<p align="left"><b>Triple Porosity Model</b>

Linear Flow Solutions for Fractured Linear Reservoirs

Constant pressure case:

$\begin{equation*}
\begin{aligned}
\frac{1}{q_{DL}} = \frac{2 \pi s}{\sqrt{s f(s)}} \left[ \frac{1+\exp{-2 \sqrt{s f(s)}} y_{De}}{1-\exp{-2 \sqrt{s f(s)}} y_{De}} \right]
\end{aligned}
\end{equation*}$

<img src="https://raw.githubusercontent.com/trmcnealy/MultiPorosityModel/master/TriplePorosity.png" width="1131" height="741" align="left">
</p>

In [1]:
#r "nuget:VegaLite.NET"
#r "nuget:Kokkos.NET"
#r "nuget:OilGas.Data"
#r "nuget:MultiPorosityModel"

Installed package VegaLite.NET version 1.3.11

Installed package OilGas.Data version 1.6.4

Installed package Kokkos.NET version 0.5.7

Installed package MultiPorosityModel version 0.3.4

In [2]:
using Kokkos;
using MultiPorosity.Models;
using MultiPorosity.Services;
using NumericalMethods.DataStorage;
using VegaLite;
using OilGas.Data.Charting;

using Newtonsoft.Json;
using Newtonsoft.Json.Serialization;

In [3]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Reflection;
using System.Runtime.CompilerServices;
using System.Runtime.InteropServices;
using System.Security;

In [4]:
int  num_threads      = 4;
int  num_numa         = 1;
int  device_id        = 0;
int  ndevices         = 1;
int  skip_device      = 9999;
bool disable_warnings = false;

InitArguments arguments = new InitArguments(num_threads,
                                            num_numa,
                                            device_id,
                                            ndevices,
                                            skip_device,
                                            disable_warnings);

In [5]:
ExecutionSpaceKind ExecutionSpace = ExecutionSpaceKind.OpenMP;

try
{
    using(ScopeGuard sg = new ScopeGuard(arguments))
    {
        Devices devices = new Devices();

        int gpuCount = devices.Gpu.Count;

        if(gpuCount > 0)
        {
            display($"Gpu Count: {gpuCount}");

            int version = devices.Gpu.First().Architecture.Version;

            display($"Gpu Arch: {devices.Gpu.First().Architecture.Name} {version}");

            if(version >= 520)
            {
                ExecutionSpace = ExecutionSpaceKind.Cuda;
            }
        }
    }
}
catch(Exception e)
{
    display(e.Message);
}

Gpu Count: 3

Gpu Arch: Maxwell 520

In [6]:
static readonly double[] timeData =
{
    1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00, 8.00, 9.00, 10.00, 11.00, 12.00, 13.00, 14.00, 15.00, 16.00, 17.00, 18.00, 19.00, 20.00, 21.00, 22.00, 23.00, 24.00, 25.00, 26.00,
    27.00, 28.00, 29.00, 30.00, 31.00, 32.00, 33.00, 34.00, 35.00, 36.00, 37.00, 38.00, 39.00, 40.00, 41.00, 42.00, 43.00, 44.00, 45.00, 46.00, 47.00, 48.00, 49.00, 50.00, 51.00,
    52.00, 53.00, 54.00, 55.00, 56.00, 57.00, 58.00, 59.00, 60.00, 61.00, 62.00, 63.00, 64.00, 65.00, 66.00, 67.00, 68.00, 69.00, 70.00, 71.00, 72.00, 73.00, 74.00, 75.00, 76.00,
    77.00, 78.00, 79.00, 80.00, 81.00, 82.00, 83.00, 84.00, 85.00, 86.00, 87.00, 88.00, 89.00, 90.00, 91.00, 92.00, 93.00, 94.00, 95.00, 96.00, 97.00, 98.00, 99.00, 100.00, 101.00,
    102.00, 103.00, 104.00, 105.00, 106.00, 107.00, 108.00, 109.00, 110.00, 111.00, 112.00, 113.00, 114.00, 115.00, 116.00, 117.00, 118.00, 119.00, 120.00, 121.00, 122.00, 123.00,
    124.00, 125.00, 126.00, 127.00, 128.00, 129.00, 130.00, 131.00
};

static readonly double[] qoData =
{
    215.00, 626.00, 650.00, 686.90, 610.70, 578.87, 603.25, 801.39, 940.65, 764.58, 705.05, 690.91, 691.24, 719.37, 771.83, 781.97, 698.35, 701.45, 679.09, 690.09, 650.20, 607.93,
    622.11, 602.76, 560.85, 571.40, 588.63, 559.52, 565.43, 528.30, 517.26, 516.94, 517.19, 516.56, 513.51, 501.84, 486.37, 468.29, 475.70, 449.74, 456.89, 466.28, 451.18, 432.44,
    439.75, 356.65, 529.26, 401.97, 390.66, 467.53, 437.38, 433.74, 372.42, 390.37, 403.21, 390.26, 363.43, 406.69, 367.72, 441.42, 410.57, 377.83, 371.07, 331.53, 401.77, 343.77,
    353.99, 397.16, 338.24, 318.80, 303.40, 323.43, 293.02, 303.23, 294.21, 293.01, 298.53, 304.64, 294.30, 294.66, 290.98, 288.29, 290.95, 288.24, 502.21, 408.63, 390.16, 374.39,
    347.30, 338.92, 313.71, 316.73, 307.36, 305.74, 292.65, 293.01, 297.31, 291.52, 282.22, 273.84, 295.63, 271.26, 278.18, 277.49, 266.27, 273.97, 265.47, 267.83, 268.14, 255.22,
    259.11, 261.13, 260.31, 254.44, 253.05, 254.78, 247.38, 253.74, 246.34, 240.89, 247.23, 244.87, 242.36, 242.69, 240.33, 269.18, 261.42, 252.41, 264.48, 260.17, 257.04
};

//static readonly double[] qwData =
//{
//    0.00, 0.00, 621.00, 639.00, 640.00, 631.00, 671.00, 822.00, 960.00, 900.00, 918.00, 710.00, 765.00, 775.00, 728.00, 630.00, 770.00, 774.00, 660.00, 608.00, 548.00, 542.00,
//    530.00, 496.00, 521.00, 453.00, 438.00, 497.00, 432.00, 416.00, 387.00, 362.00, 348.00, 388.00, 326.00, 403.00, 360.00, 380.00, 344.00, 305.00, 361.00, 353.00, 294.00, 303.00,
//    260.00, 242.00, 233.00, 213.00, 245.00, 226.00, 235.00, 258.00, 263.00, 206.00, 183.00, 204.00, 230.00, 193.00, 257.00, 231.00, 209.00, 200.00, 231.00, 246.00, 193.00, 145.00,
//    192.00, 232.00, 187.00, 196.00, 161.00, 152.00, 135.00, 118.00, 106.00, 119.00, 129.00, 111.00, 108.00, 126.00, 119.00, 119.00, 111.00, 295.00, 110.00, 153.00, 169.00, 148.00,
//    185.00, 145.00, 141.00, 145.00, 139.00, 151.00, 132.00, 150.00, 132.00, 123.00, 137.00, 139.00, 142.00, 133.00, 142.00, 130.00, 125.00, 130.00, 118.00, 119.00, 121.00, 117.00,
//    120.00, 118.00, 143.00, 115.00, 124.00, 141.00, 122.00, 118.00, 108.00, 114.00, 110.00, 113.00, 111.00, 109.00, 119.00, 127.00, 126.00, 120.00, 145.00, 160.00, 125.00
//};

static readonly double[] qgData =
{
    380.00, 380.00, 380.00, 380.00, 380.00, 380.00, 380.00, 533.00, 551.00, 537.00, 528.00, 520.00, 507.00, 503.00, 514.00, 507.00, 491.00, 311.00, 388.00, 380.00, 373.00, 430.00,
    424.00, 424.00, 409.00, 398.00, 390.00, 392.00, 389.00, 331.00, 351.00, 350.00, 351.00, 348.00, 353.00, 339.00, 337.00, 323.00, 322.00, 308.00, 325.00, 299.00, 302.00, 304.00,
    308.00, 281.00, 301.00, 290.00, 288.00, 291.00, 293.00, 299.00, 277.00, 227.00, 252.00, 269.00, 270.00, 265.00, 274.00, 289.00, 277.00, 219.00, 269.00, 250.00, 268.00, 253.00,
    248.00, 258.00, 238.00, 221.00, 219.00, 194.00, 200.00, 197.00, 93.00, 183.00, 175.00, 184.00, 181.00, 183.00, 180.00, 179.00, 177.00, 220.00, 271.00, 266.00, 254.00, 234.00,
    228.00, 218.00, 204.00, 205.00, 202.00, 200.00, 199.00, 197.00, 189.00, 188.00, 185.00, 182.00, 182.00, 180.00, 179.00, 169.00, 164.00, 176.00, 174.00, 172.00, 166.00, 164.00,
    163.00, 163.00, 161.00, 158.00, 157.00, 157.00, 153.00, 153.00, 151.00, 149.00, 148.00, 145.00, 144.00, 143.00, 143.00, 158.00, 161.00, 155.00, 166.00, 164.00, 157.00
};


static readonly double[] actualMonthlyTime =
{
    15.0,45.0,76.0,107.0,135.0,166.0,196.0,227.0,257.0,288.0,319.0,349.0,380.0,410.0,441.0,472.0,500.0,531.0,561.0,592.0,622.0,653.0,684.0,714.0,745.0,775.0,806.0,837.0,865.0,896.0,
    926.0,957.0,987.0,1018.0,1049.0,1079.0,1110.0,1140.0,1171.0,1202.0,1231.0,1262.0,1292.0,1323.0,1353.0,1384.0,1415.0,1445.0,1476.0,1506.0,1537.0,1568.0,1596.0,1627.0,1657.0,
    1688.0,1718.0,1749.0,1780.0,1810.0,1841.0,1871.0,1902.0,1933.0,1961.0,1992.0,2022.0,2053.0,2083.0,2114.0,2145.0,2175.0,2206.0,2236.0,2267.0,2298.0,
};

static readonly double[] actualMonthlyBOE =
{
    14759.0,15683.0,10436.0,9641.0,7042.0,9116.0,7062.0,6034.0,4388.0,3842.0,4305.0,3634.0,3037.0,2775.0,3576.0,2983.0,2924.0,3066.0,2730.0,713.0,1177.0,1275.0,1734.0,1409.0,
    2034.0,1804.0,1980.0,1154.0,1274.0,913.0,828.0,1542.0,1929.0,1492.0,592.0,279.0,503.0,445.0,593.0,511.0,523.0,401.0,139.0,875.0,708.0,769.0,597.0,594.0,762.0,283.0,0.0,755.0,
    837.0,626.0,104.0,608.0,661.0,721.0,546.0,533.0,552.0,529.0,488.0,484.0,440.0,475.0,288.0,44.0,191.0,386.0,635.0,285.0,1.0,53.0,756.0,461.0
};

double[] predictTimeData = Sequence.Linear(1.0, 2299.0, 1.0);
double[] predictQt;

//DataCache cachedData;

In [7]:
BoundConstraints<double>[] arg_limits = new BoundConstraints<double>[7];

// LF      = xe/nF;
// Lf      = ye/nf;

/*km*/
arg_limits[0] = new BoundConstraints<double>(0.0001,
                                             0.01);

/*kF*/
arg_limits[1] = new BoundConstraints<double>(10.0,
                                             1000.0);

/*kf*/
arg_limits[2] = new BoundConstraints<double>(0.01,
                                             10.0);

/*ye*/
arg_limits[3] = new BoundConstraints<double>(100.0,
                                             1000.0);

/*LF*/
arg_limits[4] = new BoundConstraints<double>(50.0,
                                             250.0);

/*Lf*/
arg_limits[5] = new BoundConstraints<double>(10.0,
                                             150.0);

/*sk*/
arg_limits[6] = new BoundConstraints<double>(0.0,
                                             0.0);

double result_rms;
double result_km;
double result_kF;
double result_kf;
double result_xe;
double result_ye;
double result_LF;
double result_Lf;
double result_NF;//=xe/LF
double result_Nf;//=ye/Lf
double result_sk;

In [8]:
ProductionChartCollection compare_results = new ProductionChartCollection(qoData.Length * 2);

In [9]:
using(ScopeGuard sg = new ScopeGuard(arguments))
{
    ProductionData<double> productionData = new ProductionData<double>(131);

    for(int i = 0; i < 131; ++i)
    {
        productionData[i].Time  = timeData[i];
        productionData[i].Qo    = qoData[i];
        productionData[i].Qw    = 0.0;
        productionData[i].Qg    = qgData[i];
        productionData[i].QgBoe = qgData[i] / 5.8;
        productionData[i].Qt    = productionData[i].Qo + productionData[i].QgBoe;
        
        compare_results.Add("Actual", timeData[i], productionData[i].Qt);
    }

    ReservoirProperties<double> reservoir = new ReservoirProperties<double>();
    reservoir.Length = 8106.4;
    reservoir.Width  = 2000.0;
    // reservoir.Area                        = (reservoir.Length * reservoir.Width) / 43560;
    reservoir.Thickness       = 125.0;
    reservoir.Porosity        = 0.06;
    reservoir.Permeability    = 0.002;
    reservoir.Temperature     = 275.0;
    reservoir.InitialPressure = 7000.0;

    WellProperties<double> well = new WellProperties<double>();
    well.LateralLength      = result_xe = 4800.0;
    well.BottomholePressure = 3500.0;

    FractureProperties<double> fracture = new FractureProperties<double>();
    fracture.Count        = 60;
    fracture.Width        = 0.1;
    fracture.Height       = 50.0;
    fracture.HalfLength   = 348.0;
    fracture.Porosity     = 0.20;
    fracture.Permeability = 184.0;
    fracture.Skin         = 0.0;

    NaturalFractureProperties<double> natural_fracture = new NaturalFractureProperties<double>();
    natural_fracture.Count        = 60;
    natural_fracture.Width        = 0.01;
    natural_fracture.Porosity     = 0.10;
    natural_fracture.Permeability = 1.0;

    Pvt<double> pvt = new Pvt<double>();
    pvt.OilViscosity             = 0.5;
    pvt.OilFormationVolumeFactor = 1.5;
    pvt.TotalCompressibility     = 0.00002;

    MultiPorosityData<double> mpd = new MultiPorosityData<double>();
    mpd.ProductionData            = productionData;
    mpd.ReservoirProperties       = reservoir;
    mpd.WellProperties            = well;
    mpd.FractureProperties        = fracture;
    mpd.NaturalFractureProperties = natural_fracture;
    mpd.Pvt                       = pvt;

    View<double, Cuda> actualQt = new View<double, Cuda>("actualQt",
                                                            131);

    View<double, Cuda> actualTime = new View<double, Cuda>("actualTime",
                                                            131);

    View<double, Cuda> weights = new View<double, Cuda>("weights",
                                                        131);    

    for(ulong i0 = 0; i0 < actualQt.Extent(0); ++i0)
    {
        actualQt[i0] = productionData[i0].Qt;
        actualTime[i0] = timeData[i0];        

        if(i0 < 8 || i0 >= 16 && i0 <= 17 || i0 > 66 && i0 < 90 || i0 > 125)
        {
            weights[i0] = 0.0001;
        }
        else
        {
            weights[i0] = 1.0;
        }
    }

    TriplePorosityModel<double, Cuda> tpm = new TriplePorosityModel<double, Cuda>(mpd, actualQt, actualTime);
    
    ParticleSwarmOptimizationOptions options = ParticleSwarmOptimizationOptions.Default;

    MultiPorosityResult<double, Cuda> results = tpm.Solve(options, weights, arg_limits);
    
    //cachedData = results.CachedData;
    result_rms = results.Error;
    
    View<double, Cuda> best_args = results.Args;
    
    result_km = best_args[0];
    result_kF = best_args[1];
    result_kf = best_args[2];
    result_ye = best_args[3];
    result_LF = best_args[4];
    result_Lf = best_args[5];
    result_sk = best_args[6];
    
    View<double, Cuda> simulatedQt = tpm.Calculate(actualTime,
                                                   best_args);
    
    for(ulong i0 = 0; i0 < simulatedQt.Size(); ++i0)
    {
        compare_results.Add("Simulated", timeData[i0], simulatedQt[i0]);
    }
}

In [10]:
display($"RMS: {result_rms} BOE");
display($"km: {result_km}");
display($"kF: {result_kF}");
display($"kf: {result_kf}");
display($"ye: {result_ye}");
display($"LF: {result_LF}");
display($"Lf: {result_Lf}");
display($"sk: {result_sk}");

display($"# producing Induced Fractures: {result_xe/result_LF}");
display($"# producing Natural Fractures: {result_ye/result_Lf}");

RMS: 25.431003269785887 BOE

km: 0.008038184108280978

kF: 345.7810111943381

kf: 8.161858463400872

ye: 239.27416625697373

LF: 141.90501465168887

Lf: 126.01192743545

sk: 0

# producing Induced Fractures: 33.82544310912323

# producing Natural Fractures: 1.898821572898666

In [11]:
compare_results.BuildCumulativeProduction(8);

ProductionChartData[] compare_chart_data = compare_results;

Chart chart = new Chart($"Triple Porosity Model",
                        ProductionChartCollection.DefaultSpecification(nameof(compare_chart_data)),
                        1000,
                        750);

chart

<!DOCTYPE html>
 
 
 
 
 
 
 
 
 Triple Porosity Model

In [ ]:
// Prediction

using(ScopeGuard sg = new ScopeGuard(arguments))
{
    ProductionData<double> productionData = new ProductionData<double>(131);

    for(int i = 0; i < 131; ++i)
    {
        productionData[i].Time  = timeData[i];
        productionData[i].Qo    = qoData[i];
        productionData[i].Qw    = 0.0;
        productionData[i].Qg    = qgData[i];
        productionData[i].QgBoe = qgData[i] / 5.8;
        productionData[i].Qt    = productionData[i].Qo + productionData[i].QgBoe;
    }

    ReservoirProperties<double> reservoir = new ReservoirProperties<double>();
    reservoir.Length = 6500.0;
    reservoir.Width  = 348.0;
    // reservoir.Area                        = (reservoir.Length * reservoir.Width) / 43560;
    reservoir.Thickness       = 125.0;
    reservoir.Porosity        = 0.06;
    reservoir.Permeability    = 0.002;
    reservoir.Temperature     = 275.0;
    reservoir.InitialPressure = 7000.0;

    WellProperties<double> well = new WellProperties<double>();
    well.LateralLength      = 6500.0;
    well.BottomholePressure = 3500.0;

    FractureProperties<double> fracture = new FractureProperties<double>();
    fracture.Count        = 60;
    fracture.Width        = 0.1;
    fracture.Height       = 50.0;
    fracture.HalfLength   = 348.0;
    fracture.Porosity     = 0.20;
    fracture.Permeability = 184.0;
    fracture.Skin         = 0.0;

    NaturalFractureProperties<double> natural_fracture = new NaturalFractureProperties<double>();
    natural_fracture.Count        = 60;
    natural_fracture.Width        = 0.01;
    natural_fracture.Porosity     = 0.10;
    natural_fracture.Permeability = 1.0;

    Pvt<double> pvt = new Pvt<double>();
    pvt.OilViscosity             = 0.5;
    pvt.OilFormationVolumeFactor = 1.5;
    pvt.TotalCompressibility     = 0.00002;

    MultiPorosityData<double> mpd = new MultiPorosityData<double>();
    mpd.ProductionData            = productionData;
    mpd.ReservoirProperties       = reservoir;
    mpd.WellProperties            = well;
    mpd.FractureProperties        = fracture;
    mpd.NaturalFractureProperties = natural_fracture;
    mpd.Pvt                       = pvt;

    View<double, Cuda> actualQt = new View<double, Cuda>("actualQt",
                                                            131);

    View<double, Cuda> actualTime = new View<double, Cuda>("actualTime",
                                                            131);

    View<double, Cuda> weights = new View<double, Cuda>("weights",
                                                        131);    

    for(ulong i0 = 0; i0 < actualQt.Extent(0); ++i0)
    {
        actualQt[i0] = productionData[i0].Qt;
        actualTime[i0] = timeData[i0];        

        if(i0 < 8 || i0 >= 16 && i0 <= 17 || i0 > 66 && i0 < 90 || i0 > 125)
        {
            weights[i0] = 0.0001;
        }
        else
        {
            weights[i0] = 1.0;
        }
    }

    TriplePorosityModel<double, Cuda> tpm = new TriplePorosityModel<double, Cuda>(mpd, actualQt, actualTime);    
    
    View<double, Cuda> args = new View<double, Cuda>("args", 7);

    //km: 0.0018895176183702685
    //kF: 83.69857529340902
    //kf: 4.762316834147342
    //ye: 162.8781340630174
    //LF: 148.64640077973382
    //Lf: 55.506016771447456
    //sk: 0.0

    args[0] = 0.0018895176183702685;
    args[1] = 83.69857529340902;
    args[2] = 4.762316834147342;
    args[3] = 162.8781340630174;
    args[4] = 148.64640077973382;
    args[5] = 55.506016771447456;
    args[6] = 0.0;
    
    View<double, Cuda> predictedTimeData = new View<double, Cuda>("predictedTimeData", predictTimeData.Length);
    
    for(int i0 = 0; i0 < predictTimeData.Length; ++i0)
    {
        predictedTimeData[i0] = predictTimeData[i0];
    }    

    View<double, Cuda> predictedQt = tpm.Calculate(predictedTimeData,
                                                   args);    
    
    predictQt = new double[predictedQt.Size()];
    
    for(ulong i0 = 0; i0 < predictedQt.Size(); ++i0)
    {
       predictQt[i0] = predictedQt[i0];
    }    
}

In [ ]:
ProductionChartCollection predict_results = new ProductionChartCollection(actualMonthlyBOE.Length + predictTimeData.Length);

double t;
for(int i0 = 0; i0 < predictTimeData.Length; ++i0)
{
    t = (double)predictTimeData[i0];
    predict_results.Add("Predicted", t, predictQt[i0]);
}

//predict_results.ToMonthlyProduction(new System.DateTime(2012, 10, 15), 15);

for(int i0 = 0; i0 < actualMonthlyBOE.Length; ++i0)
{
    t = (double)actualMonthlyTime[i0];
    predict_results.Add("Actual", t, actualMonthlyBOE[i0]);
}

predict_results.BuildCumulativeProduction();

In [ ]:
ProductionChartData[] predict_chart_data = predict_results;

Chart predict_chart = new Chart($"Prediction",
                        ProductionChartCollection.DefaultSpecification(nameof(predict_chart_data)),
                        1000,
                        750);

predict_chart